In [72]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

from langchain.chat_models import ChatOpenAI

# openai_api_base_address = "http://172.23.115.108:20000/v1"
openai_api_base_address = "http://192.168.3.20:20000/v1"

model = ChatOpenAI(openai_api_key = "aaabbbcccdddeeefffedddsfasdfasdf", 
    openai_api_base = openai_api_base_address,
    model_name = "chatglm3-6b-32k")

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

In [73]:
# The input schema of the chain is the input schema of its first part, the prompt.
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [74]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [75]:
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'A Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content', 'type': 'string'},
    'additional_kwargs': {'title': 'Additional Kwargs

In [76]:
# The output schema of the chain is the output schema of its last part, in this case a ChatModel, which outputs a ChatMessage
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'}],
 'definitions': {'AIMessage': {'title': 'AIMessage',
   'description': 'A Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content', 'type': 'string'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'ai',
     'enum': ['ai'],
     'type': 'string'},
    'example': {'title': 'Example', 'default': False, 'type': 'boolean'}},
   'required': ['content']},
  'HumanMessage': {'title': 'HumanMessage',
   'description': 'A Message from a human.',
   'type': 'object',
   'properties': {'content': {'title': 'Content', 'type': 'string'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'type': {'title': 'Type',
    

In [77]:
for s in chain.stream({"topic": "bears"}):
    print(s.content, end="", flush=True)

Why did the bear get in trouble with the park ranger? Because he wouldn't leave the balloons alone!

In [78]:
chain.invoke({"topic": "bears"})

AIMessage(content='Why did the bear knock on the door? To get some attention!')

In [79]:
chain.batch([{"topic": "bears"}, {"topic": "cats"}])

[AIMessage(content='Why did the bear go to the doctor? Because he was feeling熊'),
 AIMessage(content='Why did the cat bring home the can of soup? Because she wanted to get can-tastic!')]

In [80]:
chain.batch([{"topic": "bears"}, {"topic": "cats"}], config={"max_concurrency": 5})

[AIMessage(content='Why did the bear get in trouble with the teacher?\n\nBecause he was caught with his paws in the cookie jar.'),
 AIMessage(content='Why did the cat get kicked out of the bar? Because he was being a pain in the paw!')]

In [81]:
async for s in chain.astream({"topic": "bears"}):
    print(s.content, end="", flush=True)

Why did the bear go to the doctor? Because he was feeling熊。

In [82]:
await chain.ainvoke({"topic": "bears"})

AIMessage(content='Why did the bear climb a tree? Because he was a monkey!')

In [83]:
await chain.abatch([{"topic": "bears"}])

[AIMessage(content='Why did the bear crawl into the refrigerator? Because he was looking for refreshments!')]

## Streaming JSONPatch chunks
This is useful eg. to stream the JSONPatch in an HTTP server, and then apply the ops on the client to rebuild the run state there. See LangServe for tooling to make it easier to build a webserver from any Runnable.

In [84]:
from langchain.embeddings import HuggingFaceEmbeddings

# 这个也可以工作，只是使用本地embedding模型
# embedding_model = HuggingFaceEmbeddings(model_name = "moka-ai/m3e-base")

from langchain.embeddings import LocalAIEmbeddings

openai_api_base_address = "http://192.168.3.20:20000/v1"

# 这个可以工作，使用服务化embedding模型
embedding_model=LocalAIEmbeddings(openai_api_key = "aaabbbcccdddeeefffedddsfasdfasdf",  
                            openai_api_base = openai_api_base_address,
                            model = "chatglm3-6b-32k")

In [85]:

from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=embedding_model)

retriever = vectorstore.as_retriever()

retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

async for chunk in retrieval_chain.astream_log(
    "where did harrison work?", include_names=["Docs"]
):
    print("-" * 40)
    print(chunk)

----------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': 'f9dd442a-06ab-44e2-9424-0e9ab1d86a58',
            'logs': {},
            'streamed_output': []}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs',
  'value': {'end_time': None,
            'final_output': None,
            'id': '1e74861a-cd51-480e-a38e-6452948533aa',
            'metadata': {},
            'name': 'Docs',
            'start_time': '2023-11-26T15:45:25.141',
            'streamed_output_str': [],
            'tags': ['map:key:context', 'FAISS', 'LocalAIEmbeddings'],
            'type': 'retriever'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs/final_output',
  'value': {'documents': [Document(page_content='harrison worked at kensho')]}},
 {'op': 'add',
  'path': '/logs/Docs/end_time',
  'value': '2023-11-26T15:45:25.303'})
-----------------------